# Using HarmonicBalance.jl 

We explore the large-scale use of the package [HarmonicBalance.jl](https://github.com/NonlinearOscillations/HarmonicBalance.jl).  

## Import Packages

In [2]:
using Symbolics
using HarmonicBalance

## Section 1: Introduction

Later more. 

## Section 2: Some Internals of HarmonicBalance  

The package defines a mutable struct DifferentialEquation. 

In [3]:
?DifferentialEquation

search: DifferentialEquation



```julia
mutable struct DifferentialEquation
```

Holds differential equation(s) of motion and a set of harmonics to expand each variable. This is the primary input for `HarmonicBalance.jl`. After inputting the equations, the harmonics ansatz needs to be specified using `add_harmonic!`.

# Fields

  * `equations::OrderedCollections.OrderedDict{Num, Equation}`: Assigns to each variable an equation of motion.
  * `harmonics::OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}}`: Assigns to each variable a set of harmonics.

## Example

```julia-repl
julia> @variables t, x(t), y(t), ω0, ω, F, k;

# equivalent ways to enter the simple harmonic oscillator
julia> DifferentialEquation(d(x,t,2) + ω0^2 * x - F * cos(ω*t), x);
julia> DifferentialEquation(d(x,t,2) + ω0^2 * x ~ F * cos(ω*t), x);

# two coupled oscillators, one of them driven
julia> DifferentialEquation(
    [d(x,t,2) + ω0^2 * x - k*y, d(y,t,2) + ω0^2 * y - k*x] .~ [F * cos(ω*t), 0], [x,y]
);
```


Take the definition of the function flatten() from HarmonicBalance.jl. 

In [57]:
flatten(a) = collect(Iterators.flatten(a))

flatten (generic function with 1 method)

How does get_independent_variables work? 

In [12]:
?get_independent_variables

search: get_independent_variables



```julia
get_independent_variables(
    diff_eom::DifferentialEquation
) -> Any

```

Return the independent dependent variables of `diff_eom`.

---

```julia
get_independent_variables(
    eom::HarmonicEquation
) -> Vector{Num}

```

Return the independent variables (typically time) of `eom`.


In [13]:
?get_variables

search: get_variables get_cycle_variables get_independent_variables



```
get_variables(e, varlist = nothing; sort::Bool = false)
```

Return a vector of variables appearing in `e`, optionally restricting to variables in `varlist`.

Note that the returned variables are not wrapped in the `Num` type.

# Examples

```julia
julia> @variables t x y z(t);

julia> Symbolics.get_variables(x + y + sin(z); sort = true)
3-element Vector{SymbolicUtils.BasicSymbolic}:
 x
 y
 z(t)

julia> Symbolics.get_variables(x - y; sort = true)
2-element Vector{SymbolicUtils.BasicSymbolic}:
 x
 y
```

---

```julia
get_variables(diff_eom::DifferentialEquation) -> Vector{Num}

```

Return the dependent variables of `diff_eom`.

---

```julia
get_variables(eom::HarmonicEquation) -> Vector{Num}

```

Get the internal symbols of the independent variables of `eom`.


## Section 3: Build-In Example from HarmonicBalance

Run the example provided as example with the documentation of HarmonicBalance.jl. 

Store output in diff_eq0, where "0" here denotes the original system. 

In [60]:
@variables t, x(t), y(t), ω0, ω, F, k;

diff_eq0 = DifferentialEquation(
    [d(x,t,2) + ω0^2 * x - k*y, d(y,t,2) + ω0^2 * y - k*x] .~ [F * cos(ω*t), 0], [x,y]
)

System of 2 differential equations
Variables:       x(t), y(t)
Harmonic ansatz: x(t) => ;   y(t) => ;   

Differential(t)(Differential(t)(x(t))) - k*y(t) + x(t)*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)(y(t))) - k*x(t) + y(t)*(ω0^2) ~ 0


In [61]:
typeof(x)

Num

In [62]:
#typeof(diff_eq0.variables)

In [63]:
show(diff_eq0)

"equations"

OrderedCollections.OrderedDict{Num, Equation} with 2 entries:
  x(t) => Differential(t)(Differential(t)(x(t))) - k*y(t) + x(t)*(ω0^2) ~ F*cos…
  y(t) => Differential(t)(Differential(t)(y(t))) - k*x(t) + y(t)*(ω0^2) ~ 0

"harmonics"

OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 2 entries:
  x(t) => OrderedSet{Num}()
  y(t) => OrderedSet{Num}()

In [64]:
values(diff_eq0.equations)

ValueIterator for a OrderedCollections.OrderedDict{Num, Equation} with 2 entries. Values:
  Differential(t)(Differential(t)(x(t))) - k*y(t) + x(t)*(ω0^2) ~ F*cos(t*ω)
  Differential(t)(Differential(t)(y(t))) - k*x(t) + y(t)*(ω0^2) ~ 0

In [65]:
keys(diff_eq0.harmonics)

KeySet for a OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 2 entries. Keys:
  x(t)
  y(t)

In [66]:
add_harmonic!(diff_eq0, x, ω); 
diff_eq0

System of 2 differential equations
Variables:       x(t), y(t)
Harmonic ansatz: x(t) => ω;   y(t) => ;   

Differential(t)(Differential(t)(x(t))) - k*y(t) + x(t)*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)(y(t))) - k*x(t) + y(t)*(ω0^2) ~ 0


In [67]:
add_harmonic!(diff_eq0, y, ω); 
diff_eq0

System of 2 differential equations
Variables:       x(t), y(t)
Harmonic ansatz: x(t) => ω;   y(t) => ω;   

Differential(t)(Differential(t)(x(t))) - k*y(t) + x(t)*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)(y(t))) - k*x(t) + y(t)*(ω0^2) ~ 0


In [68]:
get_harmonic_equations(diff_eq0)

A set of 4 harmonic equations
Variables: u1(T), v1(T), u2(T), v2(T)
Parameters: ω, k, ω0, F

Harmonic ansatz: 
x(t) = u1(T)*cos(ωt) + v1(T)*sin(ωt)
y(t) = u2(T)*cos(ωt) + v2(T)*sin(ωt)

Harmonic equations:

-k*u2(T) + (2//1)*Differential(T)(v1(T))*ω - u1(T)*(ω^2) + u1(T)*(ω0^2) ~ F

-k*v2(T) - (2//1)*Differential(T)(u1(T))*ω - v1(T)*(ω^2) + v1(T)*(ω0^2) ~ 0

-k*u1(T) + (2//1)*Differential(T)(v2(T))*ω - u2(T)*(ω^2) + u2(T)*(ω0^2) ~ 0

-k*v1(T) - (2//1)*Differential(T)(u2(T))*ω - v2(T)*(ω^2) + v2(T)*(ω0^2) ~ 0


In [69]:
get_independent_variables(diff_eq0)

1-element Vector{Num}:
 t

In [70]:
get_variables(diff_eq0)

2-element Vector{Num}:
 x(t)
 y(t)

In [71]:
unique([x.val.arguments for x in keys(diff_eq0.equations)])

1-element Vector{Vector{Any}}:
 [t]

In [58]:
Num.(flatten(unique([x.val.arguments for x in keys(diff_eq0.equations)])))

1-element Vector{Num}:
 t

In [75]:
typeof(diff_eq0.harmonics)

OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}}

## Section 4: My Modified Example 

Make x(t) to be a two-dimensional vector. Remove y(t). 

In [46]:
 @variables t, (x(t))[1:2], ω0, ω, F, k; 

Redefine the equations and store output in diff_eq. 

In [47]:
diff_eq = DifferentialEquation(
    [d(x[1],t,2) + ω0^2 * x[1] - k*x[2], d(x[2],t,2) + ω0^2 * x[2] - k*x[1]] .~ [F * cos(ω*t), 0], [x[1],x[2]]
)

System of 2 differential equations
Variables:       (x(t))[1], (x(t))[2]
Harmonic ansatz: (x(t))[1] => ;   (x(t))[2] => ;   

Differential(t)(Differential(t)((x(t))[1])) - k*(x(t))[2] + (x(t))[1]*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)((x(t))[2])) - k*(x(t))[1] + (x(t))[2]*(ω0^2) ~ 0


In [48]:
show(diff_eq)

"equations"

OrderedCollections.OrderedDict{Num, Equation} with 2 entries:
  (x(t))[1] => Differential(t)(Differential(t)((x(t))[1])) - k*(x(t))[2] + (x(t…
  (x(t))[2] => Differential(t)(Differential(t)((x(t))[2])) - k*(x(t))[1] + (x(t…

"harmonics"

OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 2 entries:
  (x(t))[1] => OrderedSet{Num}()
  (x(t))[2] => OrderedSet{Num}()

In [49]:
values(diff_eq.harmonics)

ValueIterator for a OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 2 entries. Values:
  OrderedCollections.OrderedSet{Num}()
  OrderedCollections.OrderedSet{Num}()

### The function get_independent_variables produces counter-intuitive results 

In [40]:
get_independent_variables(diff_eq)

4-element Vector{Num}:
 x(t)
    1
 x(t)
    2

In [52]:
dynamic_vars = flatten(getfield.(diff_eq.variables, Symbol("symbols")))

LoadError: type DifferentialEquation has no field variables

In [50]:
add_harmonic!(diff_eq, x[1], ω); 
diff_eq

System of 2 differential equations
Variables:       (x(t))[1], (x(t))[2]
Harmonic ansatz: (x(t))[1] => ω;   (x(t))[2] => ;   

Differential(t)(Differential(t)((x(t))[1])) - k*(x(t))[2] + (x(t))[1]*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)((x(t))[2])) - k*(x(t))[1] + (x(t))[2]*(ω0^2) ~ 0


In [51]:
add_harmonic!(diff_eq, x[2], ω); 
diff_eq

System of 2 differential equations
Variables:       (x(t))[1], (x(t))[2]
Harmonic ansatz: (x(t))[1] => ω;   (x(t))[2] => ω;   

Differential(t)(Differential(t)((x(t))[1])) - k*(x(t))[2] + (x(t))[1]*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)((x(t))[2])) - k*(x(t))[1] + (x(t))[2]*(ω0^2) ~ 0


In [44]:
# redefine get_independent_variables 
using HarmonicBalance: get_independent_variables

In [75]:
function HarmonicBalance.get_independent_variables(diff_eq)
    return t 
end 

In [76]:
get_harmonic_equations(diff_eq)

LoadError: MethodError: no method matching String(::Expr)

[0mClosest candidates are:
[0m  String([91m::LazyString[39m)
[0m[90m   @[39m [90mBase[39m [90mstrings/[39m[90m[4mlazy.jl:80[24m[39m
[0m  String([91m::Vector{UInt8}[39m)
[0m[90m   @[39m [90mBase[39m [90mstrings/[39m[90m[4mstring.jl:67[24m[39m
[0m  String([91m::Symbol[39m)
[0m[90m   @[39m [90mBase[39m [90mstrings/[39m[90m[4mstring.jl:98[24m[39m
[0m  ...


In [79]:
edit _create_harmonic_variable

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///Users/djplahaye/mijn_onderwijs/minor-bachelor-project/2023-sediment-transport-rivers/notebooks/In[79]#1:5\[90mIn[79]:1:5[0;0m]8;;\
edit[48;2;120;70;70m _create_harmonic_variable[0;0m
[90m#   └────────────────────────┘ ── [0;0m[91mextra tokens after end of expression[0;0m